In [1]:
import numpy as np
import pandas as pd
from IPython import parallel as pr

In [2]:
c = pr.Client()

In [3]:
c.ids

[0, 1, 2, 3]

In [4]:
view = c.load_balanced_view()

In [5]:
%%px
import pandas as pd
import numpy as np
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [6]:
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [7]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df') # bring in the data frame of file paths

In [8]:
cells = pd.DataFrame()
data = np.load('./data/VIC_cells_overlap.npz') # load the soil data

# loop through the SWE data

for key in data.keys():
    cells[key] = data[key]

In [9]:
overlap = []
cells2 = np.unique(cells.cells)
for idx in cells2:
    overlap.extend(files.loc[files.indexer==idx,'flux'])

In [30]:
Q = []
years = []
idx = []

for fl,id2 in zip(overlap,cells2):
    flux = pd.read_table(fl, sep='\t', names = fluxes_columns)
    flux.index = pd.DatetimeIndex(dates)

    flux['discharge'] = flux.BF+flux.R # compute the total discharge [mm]
    flux2 = flux.groupby(by='y').sum() # compute annual totals
    flux2.reset_index(inplace=True) # reset the index

    n = len(flux2) # grab the length of the time series

    Q.extend(list(flux2.discharge))
    years.extend(list(flux2.y))
    idx.extend(list(np.repeat(id2,n))) # repeat the indexer for each year    


In [32]:
np.savez_compressed('/Volumes/data/Theo/projects/Budyko_vic/data/ground_truth_flux.npz',years=years,Q=Q,idx=idx)

In [34]:
flux2

,y,m,d,ET,R,BF,sm1,sm2,sm3,SWE,Cs,Qs,Ql,Qg,NR,PEText,PETtrc,PETsrc,discharge
0,1950,2382,5738,445.118796,23.452922,67.028953,8225.0222,25505.8511,27853.6547,2085.6075,35.4653,3999.2332,10216.3776,-649.6289,17393.3110,661.90,797.773855,528.8674,90.481874
1,1951,2382,5738,364.955290,18.828286,66.990076,8047.5441,24139.0635,27853.7345,1517.5065,33.2410,6692.9687,8831.5657,-678.6201,17562.4980,721.36,895.969763,592.7365,85.818363
2,1952,2384,5767,330.783335,8.995633,66.370108,7449.4806,21890.2784,27568.1777,575.0287,29.4093,9502.8381,8511.2008,-344.8763,19314.2568,841.16,1074.075451,696.2157,75.365741
3,1953,2382,5738,386.717926,15.865571,55.326315,8153.1950,20767.2170,22995.4588,261.9388,34.9474,7060.8032,9467.0859,-491.1205,18981.9929,763.10,945.042247,626.1152,71.191886
4,1954,2382,5738,551.228525,51.671994,92.160286,8643.9871,24041.1845,34005.8993,455.0930,43.4563,2249.0528,14252.1930,-510.2270,18583.9436,706.47,859.486846,552.7571,143.832281
5,1955,2382,5738,455.701741,15.828414,70.365244,8325.5928,25047.3832,29238.3230,926.0066,42.4917,4802.3412,11163.7218,-694.7054,18322.9939,743.58,917.034798,633.5689,86.193659
6,1956,2384,5767,391.408185,13.022112,59.040034,8233.0580,23717.4310,24528.4185,1348.5077,34.2760,7035.5896,9440.1992,-582.0950,18722.0372,751.00,926.763135,616.4043,72.062146
7,1957,2382,5738,414.060118,25.254705,54.983003,8197.6393,23485.5448,22863.1211,119.3134,37.0430,7306.2301,10955.7499,-560.9541,19605.9804,773.02,970.330668,631.1789,80.237708
8,1958,2382,5738,283.845482,4.603230,51.215560,7393.8740,19775.0250,21266.3188,1200.1657,23.0191,11271.1780,6769.7679,-486.4158,19997.4821,875.31,1129.879515,708.8277,55.818790
9,1959,2382,5738,379.413804,26.477037,53.121273,8255.8698,23190.4064,22132.0751,2268.2004,37.3949,7770.0019,9371.9210,-570.2305,19206.5274,790.67,988.402661,632.3651,79.598310
